In [ ]:
import os
from itertools import permutations, product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# magic line to have in-notebook interactive plots
%matplotlib notebook

In [ ]:
files = os.listdir("./gt_bbox")
print("num of files: ",len(files))
pool = [1, 2, 3, 4, 5, 6]
perm = []
for x in permutations(pool, 2):
    perm.append(x)
data = np.zeros(shape= (30,  6))


In [ ]:
seq_length = 165000

In [ ]:
time_seq_dic = {ele: [] for ele in perm}

cur_obj = 1
cam_seq = []
frame_seq = []
for file in files:
    if file == "Thumbs.db":
        continue
        
    obj_id = int(file[:4])
    cam_id = int(file[6])
    seq_id = int(file[8])
    frame_id = int(file[10:16])
    frame_num = frame_id + seq_length * (seq_id - 1)
    
    if obj_id != cur_obj:
#         if cur_obj == 3:
#             break;
    
        cam_seq = np.array(cam_seq)
        frame_seq = np.array(frame_seq)
        frame_seq_index = np.argsort(frame_seq)
        frame_seq_sorted = np.sort(frame_seq)
        
        #print(frame_seq_index)
        #print(cam_seq[frame_seq_index])
        #print(frame_seq_sorted)
        
        cam_tuple_before = ()
        cam_tuple_after = ()
        for i in range(len(frame_seq_sorted)-1):
            travel_time = frame_seq_sorted[i+1] - frame_seq_sorted[i]
            cam_id_before = cam_seq[frame_seq_index[i]]
            cam_id_after = cam_seq[frame_seq_index[i+1]]
                
            if cam_id_before != cam_id_after: # if camera id is not the same
                cam_tuple_after = cam_tuple_after + (cam_id_after,)
                if travel_time != 0: # if travel time is 0, just add it to tuple and contiune
                    #print(cam_tuple_before, cam_tuple_after)
                    if len(cam_tuple_before) != 0: # if tuple is not the first one
                        for prod in product(cam_tuple_before, cam_tuple_after): 
                            #print(prod)
                            if prod[0] != prod[1]: # if product is not between the same cameras
                                time_seq_dic[prod].append(travel_time)
                    cam_tuple_before = cam_tuple_after
                    cam_tuple_after = ()
                else:
                    continue
        
        cam_seq = []
        frame_seq = []
        cur_obj = obj_id
        
    cam_seq.append(cam_id)
    frame_seq.append(frame_num)
print(time_seq_dic)

In [ ]:
print(time_seq_dic)

In [ ]:
spatio_matrix = np.zeros(shape = (6,6))
df_spatio = pd.DataFrame(data=spatio_matrix, index=[1,2,3,4,5,6], columns=[1,2,3,4,5,6])
for key, value in time_seq_dic.items():
    cam_before_id = key[0]
    cam_after_id = key[1]
    df_spatio.loc[cam_before_id][cam_after_id] = len(value)
print(df_spatio)
df_spatio_normalized = df_spatio.divide(df_spatio.sum(axis=1), axis=0)
print(df_spatio_normalized)

fig = plt.figure(figsize=(11, 8))
fig.text(0.06, 0.5, 'Traffic Percentage (%)', ha='center', va='center', rotation='vertical')
y = [0.0, 0.2, 0.4, 0.6, 0.8]
x = [1, 2, 3, 4, 5, 6]
for index, row in df_spatio_normalized.iterrows():
    ax = plt.subplot(2, 3, index)
    plt.bar(row.index, row.values)
    plt.xticks(x)
    plt.yticks(y)
    vals = ax.get_yticks()
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
    plt.title("Camera " + str(index))

plt.savefig('spatio_matrix.pdf')

In [ ]:
fig = plt.figure(figsize=(11, 8))
fig.text(0.06, 0.5, 'Frequency', ha='center', va='center', rotation='vertical')

y = [0, 200, 400, 600, 800, 1200]
for key, value in time_seq_dic.items():
    print(value)
    time_arr = np.array(value) / 25
    print(time_arr)
    ax = plt.subplot(2, 3, key[0])
    plt.hist(time_arr, histtype=u'step',bins=np.linspace(0, 20000, 50))
    plt.title("Camera " + str(key[0]))

plt.savefig('temporal_matrix.pdf')

In [ ]:
# files = os.listdir("./gt_bbox")
# print("num of files: ",len(files))
# pool = [1, 2, 3, 4, 5, 6]
# comb = []
# for x in combinations(pool, 2):
#     comb.append(str(x))
# data = np.zeros(shape= (15,  6))


In [ ]:
# df = pd.DataFrame(data, columns = ["s1", "s2", "s3", "s4", "s5", "s6"],
#     index=comb)
# df.index.name = "Camera pairs"
# print(df)

In [ ]:
# print(type(df.index[0]))
# print(df.loc["(1, 2)"]["s1"])

In [ ]:
# cur_obj = 1
# seq_dic = {"s" + str(i):[] for i in range(1, 7)}
# for file in files:
#     if file == "Thumbs.db":
#         continue
#     obj_id = int(file[:4])
#     camera_id = int(file[6])
#     seq_num = file[7:9]
#     if obj_id != cur_obj:
        
#         pool = seq_dic[seq_num]
#         comb = []
#         for x in combinations(pool, 2):
#             if x[0] != x[1]:
#                 comb.append(str(x))
#         for ele in comb:
#             df.loc[ele][seq_num] = df.loc[ele][seq_num] + 1
        
#         seq_dic = {"s" + str(i):[] for i in range(1, 7)}
#         cur_obj = obj_id
#     seq_dic[seq_num].append(camera_id)
# print(df)

In [ ]:
df.to_csv("spatio_temporal_matrix.csv", encoding="utf-8")